# Exploratory Notebook

- Test Database Utility Functions
- Test API Service
- Explore Results
_____

In [24]:
%cd ../

In [2]:
%load_ext autoreload
import os
import pandas as pd

### Test Database (SQLite3)

In [3]:
%autoreload
from recommender.database import utils as db_main
from recommender.database.manager import DatabaseManager

In [ ]:
%autoreload
# Populate Data
db_main.ingest_raw_data('dev')

In [28]:
%autoreload
# Check Populated Data
df_check_tags = db_main.read_table('dev', f"select * from course_tags")
df_check_tags.head()

,course_id,course_tags
0,12-principles-animation-toon-boom-harmony-1475,2d-animation
1,2d-racing-game-series-unity-5-1312,game-design
2,2d-racing-games-unity-volume-2-1286,game-art
3,2d-racing-games-unity-volume-2-1286,digital-painting
4,2d-racing-games-unity-volume-2-1286,image-editing


In [36]:
%autoreload
# Create Table for Similarity Results
db_main.create_ranking_table('dev','test_ranking_table', top_n=5)

In [30]:
# Sample Data
df_test = pd.DataFrame({'user_handle':['110','110','111','111'],
                        'user_match': ['112','113','157','145'],
                        'similarity': ['80.2','20.8','52.0','48.0']})

In [31]:
%autoreload
# Write Similarty Results to Table
db_main.write_table('dev','test_table',df_test)

In [33]:
%autoreload
# Read from Table
users = '110'
db_main.read_table('dev', f"select * from test_table where user_handle = {users}")

,user_handle,user_match,similarity
0,110,112,80.2
1,110,113,20.8


##### Add Index on Results Table (user_ranking)

In [ ]:
# Index user handle
with DatabaseManager('dev') as conn:
        sql_table = f"""CREATE UNIQUE INDEX user_handle_index ON ranking (user_handle)"""
        cur = conn.cursor()
        cur.execute(sql_table)

### Test Flask API

```
python runserver.py
```


**Python Script**

In [ ]:
!export DATABASE_ENV=dev

In [ ]:
!export RESULTS_TABLE=user_ranking

In [112]:
%autoreload
os.environ['DATABASE_ENV'] = 'dev'
os.environ['RESULTS_TABLE'] = 'user_ranking'
from server.api import similarity
similarity(user_id='110')

{
  "110": [
    "{'user': 5182, 'score': 0.4331}", 
    "{'user': 5029, 'score': 0.4159}", 
    "{'user': 1133, 'score': 0.4083}", 
    "{'user': 7477, 'score': 0.4043}", 
    "{'user': 2867, 'score': 0.3992}"
  ]
}



**Make post request using CURL from command line**

In [12]:
!curl -X GET -H "Content-type: application/json" -d "{\"user_handle\":\"110\"}" "http://0.0.0.0:5000/api/similarity/"

{
  "110": [
    "{'user': 5182, 'score': 0.4331}", 
    "{'user': 5029, 'score': 0.4159}", 
    "{'user': 1133, 'score': 0.4083}", 
    "{'user': 7477, 'score': 0.4043}", 
    "{'user': 2867, 'score': 0.3992}"
  ]
}


### Results

In [11]:
# User content
user_assesments = pd.read_csv(os.path.join('data','user_assessment_scores.csv'))
user_interest = pd.read_csv(os.path.join('data','user_interests.csv'))
user_course_views = pd.read_csv(os.path.join('data','user_course_views.csv'))
course_tags = pd.read_csv(os.path.join('data','course_tags.csv'))

In [15]:
# Input User == 2
%autoreload
db_main.read_table('dev', f"select * from user_ranking where user_handle = 2")

,1,2,3,4,5,user_handle
0,"{'user': 6743, 'score': 0.4869}","{'user': 8635, 'score': 0.4592}","{'user': 1175, 'score': 0.4509}","{'user': 9014, 'score': 0.4464}","{'user': 1882, 'score': 0.4449}",2


In [17]:
user_interest[user_interest['user_handle'] == 2]

,user_handle,interest_tag,date_followed
140,2,zbrush,2017-07-03 04:37:31
141,2,maya,2017-07-03 04:37:31
142,2,3ds-max,2017-07-03 04:37:31
143,2,adobe,2017-07-03 04:37:31


In [19]:
user_interest[user_interest['user_handle'] == 6743].head(10)

,user_handle,interest_tag,date_followed
198981,6743,3d-animation,2017-07-13 06:16:36
198982,6743,3d-lighting,2017-07-13 06:16:36
198983,6743,adobe,2017-07-13 06:16:36
198984,6743,3d-texturing,2017-07-13 06:16:36
198985,6743,maya,2017-07-13 06:16:36
198986,6743,3d-modeling,2017-07-13 06:16:36
198987,6743,javascript,2017-07-13 06:16:36
198988,6743,3d-dynamics,2017-07-13 06:16:36
198989,6743,3ds-max,2017-07-13 06:16:36
198990,6743,3d-rigging,2017-07-13 06:16:36


In [20]:
user_assesments[user_assesments['user_handle'] == 2]

,user_handle,assessment_tag,user_assessment_date,user_assessment_score
524,2,photoshop,2016-09-23 16:59:45,139


In [21]:
user_assesments[user_assesments['user_handle'] == 6743]

,user_handle,assessment_tag,user_assessment_date,user_assessment_score
4235,6743,after-effects-cc,2017-07-14 11:55:47,89
4236,6743,graphic-design,2017-07-14 12:26:32,150
4237,6743,illustrator,2017-07-14 12:09:24,145
4238,6743,maya-core-skills,2017-07-14 12:02:35,140
4239,6743,photoshop-photo-manipulation,2017-07-14 12:19:58,98
4240,6743,video-production,2017-07-17 16:35:48,111
4241,6743,zbrush-core-skills,2017-07-14 12:13:31,32


In [22]:
user_course_views[user_course_views['user_handle'] == 2]

,user_handle,view_date,course_id,author_handle,level,view_time_seconds
8,2,2017-05-01,arnold-maya-fundamentals,273,Beginner,3277
9,2,2017-05-08,animated-web-social-media-banners-photoshop-fl...,62,Advanced,1996
10,2,2017-05-08,arnold-maya-fundamentals,273,Beginner,2612
11,2,2017-05-09,arnold-maya-fundamentals,273,Beginner,2142
12,2,2017-05-11,design-2d-game-level-illustrator-2113,640,Advanced,2131
...,...,...,...,...,...,...
156,2,2017-11-14,maya-character-environment-fx-bifrost-xgen,899,Intermediate,82
157,2,2017-11-14,maya-dynamics-common-effects-techniques,442,Intermediate,543
158,2,2017-11-14,maya-nparticle-fundamentals,720,Beginner,1355
159,2,2017-11-14,transformation-motion-graphics-maya-after-effe...,353,Advanced,69


In [23]:
user_course_views[user_course_views['user_handle'] == 6743]

,user_handle,view_date,course_id,author_handle,level,view_time_seconds
167970,6743,2017-05-05,procedural-rigging-python-maya-2283,1021,Advanced,637
167971,6743,2017-05-05,introductionaefor-motion-designers-1019,1058,Beginner,90
167972,6743,2017-05-23,sculpting-budget-sculptris-blender-2110,405,Advanced,627
167973,6743,2017-05-25,sculpting-budget-sculptris-blender-2110,405,Advanced,627
167974,6743,2017-07-13,drawing-fundamentals,831,Beginner,3875
167975,6743,2017-07-13,procedural-rigging-python-maya-2283,1021,Advanced,81
167976,6743,2017-07-14,introduction-unreal-engine-4-1609,489,Beginner,80
167977,6743,2017-07-14,time-management-technical-professionals,262,Beginner,905
167978,6743,2017-08-23,rigging-human-face-maya-27,239,Intermediate,1445
167979,6743,2017-08-24,rigging-human-face-maya-27,239,Intermediate,4429
